# ETL: Data Processing

Raw Dataframe > mapping > to a no nest JSON

In [4]:


#from petropylib import petroclient,utils
from pandas import json_normalize
import numpy as np
# NEW IMPORTS
import os
import time
import json
from pathlib import Path
import pandas as pd
import requests
import datetime
from collections import defaultdict

import json

In [5]:
#import function files
#get home directory
home_dir = Path.home()

os.chdir(home_dir.joinpath('Documents/repos/ETL/src/'))
from functions_no_nest import *
from mapping_no_nest import *

#get home directory
current_dir = Path.cwd()
home_dir = Path.home()

## Functions

In [6]:
def remove_nested_none_other(row):
    """
    remove none from dictonary for nested values
    """
    
    # remove nested None from value or lat/lon

    # removing None from other
    emptyotherKeyList = []
    row['other']
    for k,v in row['other'].items():
        #print(k,v)
        if v == None:
            #print(k,v)
            emptyotherKeyList.append(k)
    emptyotherKeyList = list(dict.fromkeys(emptyotherKeyList))
    [row['other'].pop(key) for key in emptyotherKeyList]
    
    return row

def replace_nones(row):
    """
    remove none from dictonary for nested values
    """
    numericList = ['elevation','daysOn','prodMonthNo']
    dateList = ['firstProductionDate','spudDate','completionDate']
    # remove nested None from value or lat/lon
    emptyKeyList = []
    for k,v in row.items():
        if v == None:
            emptyKeyList.append(k)
    #remove duplicates from list
    emptyKeyList = list(dict.fromkeys(emptyKeyList))
    
    
    for key in emptyKeyList:
        if key in numericList:
            row[key] = np.nan
        if key in dateList:
            row[key] = None
    return row

def replace_nested_value(row):
    """
    remove none from dictonary for nested values
    """
    
    # remove nested None from value or lat/lon
    emptyKeyList = []
    for k,v in row.items():
        if isinstance(v, dict):
            #print(k,v)
            for k2,v2 in v.items():
                #print(k2,v2)
                if k2 == "value":
                    if v2 == None:
                        
                        #print("nested")
                        #print(k,v)
                        emptyKeyList.append(k)
    #remove duplicates from list
    emptyKeyList = list(dict.fromkeys(emptyKeyList))

    for key in emptyKeyList:
        row[key]['value']=np.nan
        
    return row


def replace_nested_loc(row):
    """
    remove none from dictonary for nested values
    """
    
    # remove nested None from value or lat/lon
    emptyKeyList = []
    for k,v in row.items():
        if isinstance(v, dict):
            #print(k,v)
            for k2,v2 in v.items():
                #print(k2,v2)
                if k2 == 'latitude' or k2 == 'longitude':
                    if v2 == None:
                        #print("nested")
                        #print(k,v)
                        emptyKeyList.append(k)
    #remove duplicates from list
    emptyKeyList = list(dict.fromkeys(emptyKeyList))
    for key in emptyKeyList:
        #print(row[key])
        row[key]['latitude'] =np.nan
        row[key]['longitude'] =np.nan
        row[key]['coordinates'] = [np.nan,np.nan]
    return row

In [7]:
def remove_none(row):
    row = {k: v for k, v in row.items() if v is not None}
    return row

## IMPORT RAW DATA

In [8]:
path = home_dir.joinpath('Documents/repos/ETL/data/raw/01_di_wellheader_ROBERTSON.csv')
exportpathWH = home_dir.joinpath('Documents/repos/ETL/data/processed/01_wellInfo.csv')
dfWH = pd.read_csv(path,sep=',', dtype={'API14':str})

path = home_dir.joinpath('Documents/repos/ETL/data/raw/02_di_prodMonthly_ROBERTSON.csv')
exportpathProd = home_dir.joinpath('Documents/repos/ETL/data/processed/02_prodMonthly.csv')
dfProd = pd.read_csv(path,sep=',', dtype={'Api14':str})

In [9]:
#convert Nans to None

dfWH = dfWH.where(dfWH.notnull(), None)
dfProd = dfProd.where(dfProd.notnull(), 0)


## Process Data

In [12]:
mapList = []
for index, row in dfWH.iterrows():
    mapping = well_rollups_map(row)
    
    mapping = remove_none(mapping)
    #mapping = replace_nones(mapping)
        
    #mapping = replace_nested_value(mapping)
        
    #mapping = replace_nested_loc(mapping)
        
    mapping = remove_nested_none_other(mapping)
    
    mapList.append(mapping)

In [13]:
well_info_list = [well['wellId'] for well in mapList]
well_info_list

['42395800200000',
 '42395800170000',
 '42395800160000',
 '42395800150000',
 '42395800140000',
 '42395800130000',
 '42395800120000',
 '42395800110000',
 '42395800100000',
 '42395800090000',
 '42395800080000',
 '42395800070000',
 '42395800060000',
 '42395800050000',
 '42395800040000',
 '42395800030000',
 '42395800020000',
 '42395800010000',
 '42395800000000',
 '42395700340000',
 '42395682810000',
 '42395366480000',
 '42395365100000',
 '42395335440000',
 '42395330340000',
 '42395328780000',
 '42395320610000',
 '42395320580000',
 '42395320570000',
 '42395320560000',
 '42395320540000',
 '42395320530000',
 '42395320520000',
 '42395320510000',
 '42395320500000',
 '42395320490000',
 '42395320480000',
 '42395320470000',
 '42395320460000',
 '42395320450000',
 '42395320440000',
 '42395320430000',
 '42395320420000',
 '42395320410000',
 '42395320400000',
 '42395320390000',
 '42395320380000',
 '42395320370000',
 '42395320360000',
 '42395320350000',
 '42395320340000',
 '42395320330000',
 '4239532032

In [9]:
#exportpathWH_JSON = home_dir.joinpath('Documents/repos/ETL/dataProcessing/02_data_processed/01_wellInfo.json')
exportpathWH_JSON = home_dir.joinpath('Documents/repos/assetAnalysis/data/01_wellInfo.json')
with open(exportpathWH_JSON, 'w') as fp:
    json.dump(mapList, fp)

In [14]:
mapList = []
for index, row in dfProd.iterrows():
    mapping = well_production_details_map(row)
    
    mapping = remove_none(mapping)
    #mapping = replace_nones(mapping)
        
    #mapping = replace_nested_value(mapping)
        
    #mapping = replace_nested_loc(mapping)
        
    mapping = remove_nested_none_other(mapping)
    
    mapList.append(mapping)

In [15]:
dfprod = pd.json_normalize(mapList)
dfprod

,wellId,prodDate,daysOn,oilVol,gasVol,waterVol,oilRate,gasRate,waterRate,oilCum,gasCum,waterCum,prodMonthNo,other.UpdatedDate_DI,other.DeletedDate_DI,other.CreatedDate_DI,other.Api10
0,42395320460000,2020-01-01T00:00:00Z,0.0,8312.0,980,0.0,268.13,31.61,0.0,12834.0,3307,0.0,2,2020-03-19T16:13:24.337Z,0.0,2020-03-19T16:13:24.337Z,4239532046
1,42395320460000,2019-12-01T00:00:00Z,0.0,4522.0,2327,0.0,145.87,75.06,0.0,4522.0,2327,0.0,1,2020-02-18T12:47:13.04Z,0.0,2020-02-18T12:47:13.04Z,4239532046
2,42395320450000,2020-01-01T00:00:00Z,0.0,12027.0,1296,0.0,387.97,41.81,0.0,18904.0,2429,0.0,2,2020-03-19T16:13:24.337Z,0.0,2020-03-19T16:13:24.337Z,4239532045
3,42395320450000,2019-12-01T00:00:00Z,0.0,6877.0,1133,0.0,221.84,36.55,0.0,6877.0,1133,0.0,1,2020-02-18T12:47:13.04Z,0.0,2020-02-18T12:47:13.04Z,4239532045
4,42395320400000,2020-01-01T00:00:00Z,0.0,8795.0,2674,0.0,283.71,86.26,0.0,87337.0,15313,0.0,8,2020-03-19T16:13:24.337Z,0.0,2020-03-19T16:13:24.337Z,4239532040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46241,42395303670000,1991-08-01T00:00:00Z,0.0,992.0,9642,0.0,32.00,311.03,0.0,6407.0,10309,0.0,5,2019-01-17T17:37:34.393Z,0.0,2018-07-31T16:26:26.143Z,4239530367
46242,42395303670000,1991-07-01T00:00:00Z,0.0,239.0,93,0.0,7.71,3.00,0.0,5415.0,667,0.0,4,2019-01-17T17:37:34.393Z,0.0,2018-07-31T16:26:26.143Z,4239530367
46243,42395303670000,1991-06-01T00:00:00Z,0.0,3893.0,339,0.0,129.77,11.30,0.0,5176.0,574,0.0,3,2019-01-17T17:37:34.393Z,0.0,2018-07-31T16:26:26.143Z,4239530367
46244,42395303670000,1991-05-01T00:00:00Z,0.0,1263.0,234,0.0,40.74,7.55,0.0,1283.0,235,0.0,2,2019-01-17T17:37:34.393Z,0.0,2018-07-31T16:26:26.143Z,4239530367


In [63]:
dfprod['prodDate'] = pd.to_datetime(dfprod['prodDate'])
dfprod.head(2)

,wellId,prodDate,daysOn,oilVol,gasVol,waterVol,oilRate,gasRate,waterRate,oilCum,gasCum,waterCum,prodMonthNo,other.UpdatedDate_DI,other.DeletedDate_DI,other.CreatedDate_DI,other.Api10
0,42395320460000,2020-01-01 00:00:00+00:00,0.0,8312.0,980,0.0,268.13,31.61,0.0,12834.0,3307,0.0,2,2020-03-19T16:13:24.337Z,0.0,2020-03-19T16:13:24.337Z,4239532046
1,42395320460000,2019-12-01 00:00:00+00:00,0.0,4522.0,2327,0.0,145.87,75.06,0.0,4522.0,2327,0.0,1,2020-02-18T12:47:13.04Z,0.0,2020-02-18T12:47:13.04Z,4239532046


In [65]:
dfminDate = dfprod.groupby(['wellId'])['prodDate'].min().reset_index()
dfminDate

,wellId,prodDate
0,42395303670000,1991-04-01 00:00:00+00:00
1,42395303680000,1991-06-01 00:00:00+00:00
2,42395303700000,1991-11-01 00:00:00+00:00
3,42395303820000,1992-03-01 00:00:00+00:00
4,42395303840000,1992-05-01 00:00:00+00:00
...,...,...
310,42395320380000,2019-10-01 00:00:00+00:00
311,42395320390000,2019-06-01 00:00:00+00:00
312,42395320400000,2019-06-01 00:00:00+00:00
313,42395320450000,2019-12-01 00:00:00+00:00


In [68]:
dfminDate['year'] = dfminDate['prodDate'].dt.year

In [74]:
dfminDate = dfminDate[dfminDate['year']>2014]
list(dfminDate.wellId)

['42395319460000',
 '42395319610000',
 '42395319680000',
 '42395319690000',
 '42395319730000',
 '42395319740000',
 '42395319780000',
 '42395319810000',
 '42395319880000',
 '42395319890000',
 '42395319900000',
 '42395319940000',
 '42395319950000',
 '42395319960000',
 '42395319970000',
 '42395319980000',
 '42395320000000',
 '42395320020000',
 '42395320050000',
 '42395320060000',
 '42395320100000',
 '42395320120000',
 '42395320130000',
 '42395320140000',
 '42395320150000',
 '42395320160000',
 '42395320170000',
 '42395320210000',
 '42395320220000',
 '42395320230000',
 '42395320250000',
 '42395320270000',
 '42395320290000',
 '42395320300000',
 '42395320330000',
 '42395320340000',
 '42395320360000',
 '42395320370000',
 '42395320380000',
 '42395320390000',
 '42395320400000',
 '42395320450000',
 '42395320460000']

In [75]:
well_prod_id_list =['42395319460000',
 '42395319610000',
 '42395319680000',
 '42395319690000',
 '42395319730000',
 '42395319740000',
 '42395319780000',
 '42395319810000',
 '42395319880000',
 '42395319890000',
 '42395319900000',
 '42395319940000',
 '42395319950000',
 '42395319960000',
 '42395319970000',
 '42395319980000',
 '42395320000000',
 '42395320020000',
 '42395320050000',
 '42395320060000',
 '42395320100000',
 '42395320120000',
 '42395320130000',
 '42395320140000',
 '42395320150000',
 '42395320160000',
 '42395320170000',
 '42395320210000',
 '42395320220000',
 '42395320230000',
 '42395320250000',
 '42395320270000',
 '42395320290000',
 '42395320300000',
 '42395320330000',
 '42395320340000',
 '42395320360000',
 '42395320370000',
 '42395320380000',
 '42395320390000',
 '42395320400000',
 '42395320450000',
 '42395320460000']

In [ ]:
# for wellId in well_prod_id_list:
#     if wellId in mapList:
        

In [60]:
# well_prod_list = [well['wellId'] for well in mapList]
# well_prod_list

['42395320460000',
 '42395320460000',
 '42395320450000',
 '42395320450000',
 '42395320400000',
 '42395320400000',
 '42395320400000',
 '42395320400000',
 '42395320400000',
 '42395320400000',
 '42395320400000',
 '42395320400000',
 '42395320390000',
 '42395320390000',
 '42395320390000',
 '42395320390000',
 '42395320390000',
 '42395320390000',
 '42395320390000',
 '42395320390000',
 '42395320380000',
 '42395320380000',
 '42395320380000',
 '42395320370000',
 '42395320370000',
 '42395320370000',
 '42395320370000',
 '42395320370000',
 '42395320370000',
 '42395320360000',
 '42395320360000',
 '42395320360000',
 '42395320360000',
 '42395320360000',
 '42395320360000',
 '42395320340000',
 '42395320340000',
 '42395320340000',
 '42395320330000',
 '42395320330000',
 '42395320330000',
 '42395320330000',
 '42395320330000',
 '42395320300000',
 '42395320300000',
 '42395320300000',
 '42395320300000',
 '42395320300000',
 '42395320300000',
 '42395320300000',
 '42395320300000',
 '42395320300000',
 '4239532030

In [61]:
# len(set(well_prod_list))

315

In [ ]:
for data 

In [37]:
start = time.time()
#mySet = set(well_obj_list).intersection(dict_keys_list)
mySet = np.intersect1d(well_info_list, well_prod_list)

print('Duration: {} seconds'.format(time.time() - start))
myProdWells = mySet[0:20]
myProdWells

Duration: 0.07846951484680176 seconds


array(['42395303670000', '42395303680000', '42395303700000',
       '42395303820000', '42395303840000', '42395303870000',
       '42395303880000', '42395303890000', '42395303930000',
       '42395303940000', '42395304000000', '42395304020000',
       '42395304040000', '42395304050000', '42395304060000',
       '42395304070000', '42395304090000', '42395304110000',
       '42395304120000', '42395304130000'], dtype='<U14')

In [76]:
prod_data_list = []
for data in mapList:
    if data['wellId'] in well_prod_id_list:
        prod_data_list.append(data)
len(prod_data_list)

1095

In [36]:
len(prod_data_list)

7948

In [12]:
# df = pd.json_normalize(mapList)
# df.tail()

In [77]:
#exportpathWH_JSON = home_dir.joinpath('Documents/repos/ETL/dataProcessing/02_data_processed/01_wellInfo.json')
exportpathProd_JSON = home_dir.joinpath('Documents/repos/assetAnalysis/data/02_prodMonthly.json')
with open(exportpathProd_JSON, 'w') as fp:
    json.dump(prod_data_list, fp)

In [78]:
mapList2 = []
for index, row in dfWH.iterrows():
    mapping = well_rollups_map(row)
    
    mapping = remove_none(mapping)
    #mapping = replace_nones(mapping)
        
    #mapping = replace_nested_value(mapping)
        
    #mapping = replace_nested_loc(mapping)
        
    mapping = remove_nested_none_other(mapping)
    
    mapList2.append(mapping)

In [81]:
well_data_list = []
for data in mapList2:
    if data['wellId'] in well_prod_id_list:
        well_data_list.append(data)
len(well_data_list)

43

In [43]:
wellsNoProd = mapList2[0:10]

In [82]:
myNewWells = wellsNoProd + well_data_list
len(myNewWells)

53

In [83]:
_ = [well['wellId'] for well in myNewWells]
len(list(set(_)))

53

In [84]:
#exportpathWH_JSON = home_dir.joinpath('Documents/repos/ETL/dataProcessing/02_data_processed/01_wellInfo.json')
exportpathWH_JSON = home_dir.joinpath('Documents/repos/assetAnalysis/data/01_wellInfo.json')
with open(exportpathWH_JSON, 'w') as fp:
    json.dump(myNewWells, fp)